#Часть 1

Скачаем и установим саму графовую БД Nebula Graph при помощи контейнеров Docker. Для этого клонируем репозиторий с версией 3.3.0.

In [1]:
!chcp 65001
!git clone -b release-3.3 https://github.com/vesoft-inc/nebula-docker-compose.git
!docker-compose --project-directory nebula-docker-compose up -d

Active code page: 65001


Cloning into 'nebula-docker-compose'...
Container nebula-docker-compose-metad1-1  Recreate
Container nebula-docker-compose-metad2-1  Recreate
Container nebula-docker-compose-metad0-1  Recreate
Container nebula-docker-compose-metad0-1  Recreated
Container nebula-docker-compose-metad2-1  Recreated
Container nebula-docker-compose-metad1-1  Recreated
Container nebula-docker-compose-storaged1-1  Recreate
Container nebula-docker-compose-storaged2-1  Recreate
Container nebula-docker-compose-storaged0-1  Recreate
Container nebula-docker-compose-storaged2-1  Recreated
Container nebula-docker-compose-storaged0-1  Recreated
Container nebula-docker-compose-storaged1-1  Recreated
Container nebula-docker-compose-graphd-1  Recreate
Container nebula-docker-compose-graphd2-1  Recreate
Container nebula-docker-compose-graphd1-1  Recreate
Container nebula-docker-compose-graphd-1  Recreated
Container nebula-docker-compose-console-1  Recreate
Container nebula-docker-compose-graphd1-1  Recreated
Container ne

Проверим, что контейнеры запущены.

In [2]:
!docker ps

CONTAINER ID   IMAGE                               COMMAND                  CREATED          STATUS                            PORTS                                                                                                  NAMES
145648f2389c   vesoft/nebula-console:nightly       "sh -c 'for i in `se…"   9 seconds ago    Up Less than a second                                                                                                                    nebula-docker-compose-console-1
741f945ab6c8   vesoft/nebula-graphd:v3.3.0         "/usr/local/nebula/b…"   18 seconds ago   Up 2 seconds (health: starting)   0.0.0.0:49493->9669/tcp, 0.0.0.0:49491->19669/tcp, 0.0.0.0:49492->19670/tcp                            nebula-docker-compose-graphd2-1
1e16971c1c96   vesoft/nebula-graphd:v3.3.0         "/usr/local/nebula/b…"   19 seconds ago   Up 2 seconds (health: starting)   0.0.0.0:49496->9669/tcp, 0.0.0.0:49494->19669/tcp, 0.0.0.0:49495->19670/tcp                            nebula-doc

Теперь скачаем и таким же образом из контейнера поднимем Nebula Graph Studio на 7001 порту. Это понадобится для простой и поятной визуализации результатов.

In [3]:
!git clone -b release-3.6 https://github.com/vesoft-inc/nebula-graph-studio
!docker-compose --project-directory nebula-graph-studio/deployment/docker up -d

Cloning into 'nebula-graph-studio'...
Container docker-web-1  Running


Проверим статус.

In [4]:
!docker-compose --project-directory nebula-graph-studio/deployment/docker ps

NAME                COMMAND             SERVICE             STATUS              PORTS
docker-web-1        "./server"          web                 running             0.0.0.0:7001->7001/tcp


Всё поднялось и работает.

Далее подключим некоторые необходимые библиотеки и получим датасет. Для этого скачаем его и сразу запишем в DataFrame. Выведем его размер.

In [ ]:
%pip install pandas requests prettytable

In [4]:
import pandas as pd
import requests
import urllib
import json


folder_url = 'https://disk.yandex.ru/d/s6wWqd8Ol_5IvQ'
file_url = 'data_test.csv'
url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download' + '?public_key=' + urllib.parse.quote(folder_url) + '&path=/' + urllib.parse.quote(file_url)

r = requests.get(url) # запрос ссылки на скачивание
h = json.loads(r.text)['href'] # 'парсинг' ссылки на скачивание

df = pd.read_csv(h, sep=';')
df.shape


(5000, 3)

#Часть 2

Начинаем работу с самой БД. Для этого подключимся к ней и создадим пространство для данных(Space). Сразу выберем его для использования и создадим схему даных, она будет состоять из одного тега для участника события с его именем внутри(person) и одного ребра для самого события с его идентификатором(event).

In [7]:
from nebula3.gclient.net import ConnectionPool
from nebula3.Config import Config

# define a config
config = Config()
config.max_connection_pool_size = 10
# init connection pool
connection_pool = ConnectionPool()
# if the given servers are ok, return true, else return false
ok = connection_pool.init([('127.0.0.1', 9669)], config)

# option 1 control the connection release yourself
# get session from the pool
session = connection_pool.get_session('root', 'nebula')

# create and select space
db_creation = session.execute(
            'CREATE SPACE IF NOT EXISTS test(vid_type=FIXED_STRING(300)); USE test;'
            'CREATE TAG IF NOT EXISTS person(name string);'
            'CREATE EDGE event(event_id int);'
        )

'Пространство успешно создано' if db_creation.is_succeeded() else 'Возникла ошибка ' + db_creation.error_msg()


'Пространство успешно создано'

Создадим индексы.

In [8]:
idx = session.execute('CREATE TAG INDEX IF NOT EXISTS person_index on person();'
                'CREATE EDGE INDEX IF NOT EXISTS edge_index on event();')

'Индексы успешно созданы' if idx.is_succeeded() else 'Возникла ошибка' + idx.error_msg()

'Индексы успешно созданы'

In [9]:
from typing import Dict

import pandas as pd
import prettytable
from nebula3.data.DataObject import Value, ValueWrapper
from nebula3.data.ResultSet import ResultSet


def result_to_df(result: ResultSet) -> pd.DataFrame:
    """
    build list for each column, and transform to dataframe
    """
    assert result.is_succeeded(), result.error_msg()
    columns = result.keys()
    d: Dict[str, list] = {}
    for col_num in range(result.col_size()):
        col_name = columns[col_num]
        col_list = result.column_values(col_name)
        d[col_name] = [x.cast() for x in col_list]
    return pd.DataFrame.from_dict(d, orient='columns')

cast_as = {
    Value.NVAL: "as_null",
    Value.__EMPTY__: "as_empty",
    Value.BVAL: "as_bool",
    Value.IVAL: "as_int",
    Value.FVAL: "as_double",
    Value.SVAL: "as_string",
    Value.LVAL: "as_list",
    Value.UVAL: "as_set",
    Value.MVAL: "as_map",
    Value.TVAL: "as_time",
    Value.DVAL: "as_date",
    Value.DTVAL: "as_datetime",
    Value.VVAL: "as_node",
    Value.EVAL: "as_relationship",
    Value.PVAL: "as_path",
    Value.GGVAL: "as_geography",
    Value.DUVAL: "as_duration",
}

def customized_cast_with_dict(val: ValueWrapper):
    _type = val._value.getType()
    method = cast_as.get(_type)
    if method is not None:
        return getattr(val, method, lambda *args, **kwargs: None)()
    raise KeyError("No such key: {}".format(_type))

def print_resp(resp: ResultSet):
    assert resp.is_succeeded(), resp.error_msg()
    output_table = prettytable.PrettyTable()
    output_table.field_names = resp.keys()
    for recode in resp:
        value_list = []
        for col in recode:
            val = customized_cast_with_dict(col)
            value_list.append(val)
        output_table.add_row(value_list)
    print(output_table)

Занесём данные об участниках и событиях в базу.

In [10]:
for i in list(df.index):
    event_id = int(df.iloc[i,0])
    person1 = df.iloc[i,1]
    person2 = df.iloc[i,2]
    queries = f'INSERT VERTEX IF NOT EXISTS person(name) VALUES "{person1}":("{person1}"), "{person2}":("{person2}");' + \
                f'INSERT EDGE IF NOT EXISTS event(event_id) VALUES "{person1}"->"{person2}":({event_id});'

    res = session.execute(queries)
    assert res.is_succeeded(), res.error_msg()

Настроим красивый вывод результатов запросов.

Выполним несколько запросов.
Сначала выведем первых 50 учасников и посчитаем их общее количество.

In [11]:
print_resp(session.execute('MATCH (p:person) RETURN p AS person LIMIT 50'))
print("Учасников", session.execute('MATCH (p:person) RETURN count(p)').row_values(0)[0])

+----------------------------------------------------------------------------------------+
|                                         person                                         |
+----------------------------------------------------------------------------------------+
|            ("Абахова Кира Егоровна" :person{name: "Абахова Кира Егоровна"})            |
|         ("Агеносов Леонид Егорович" :person{name: "Агеносов Леонид Егорович"})         |
|      ("Алатырцева Дарья Эдуардовна" :person{name: "Алатырцева Дарья Эдуардовна"})      |
|           ("Андер Мария Николаевна" :person{name: "Андер Мария Николаевна"})           |
|           ("Андрусива Яна Олеговна" :person{name: "Андрусива Яна Олеговна"})           |
|      ("Антоненков Роберт Федорович" :person{name: "Антоненков Роберт Федорович"})      |
|         ("Аптраков Кирилл Глебович" :person{name: "Аптраков Кирилл Глебович"})         |
|           ("Атаева Мария Вадимовна" :person{name: "Атаева Мария Вадимовна"})           |

Получилось меньше, чем записей в датасете, значит некоторые имена повторяются.

Выведем первые 50 связей, количество уникальных из всех и проверим на наличие петель.

In [12]:
print_resp(session.execute('MATCH ()-[e:event]->() RETURN e AS event LIMIT 50;'))
print("Уникальных событий", session.execute('MATCH ()-[e:event]-() RETURN count(distinct e.event_id)').row_values(0)[0])

+---------------------------------------------------------------------------------------------------------+
|                                                  event                                                  |
+---------------------------------------------------------------------------------------------------------+
|        ("Арсений Никита Артурович")-[:event@0{event_id: 747373}]->("Величкина Карина Степановна")       |
|         ("Кравец Владимир Кириллович")-[:event@0{event_id: 67252}]->("Явдохина Алла Германовна")        |
|          ("Урюпин Эдуард Маратович")-[:event@0{event_id: 106645}]->("Мечиева Марина Андреевна")         |
|          ("Сулоев Роман Михаилович")-[:event@0{event_id: 732266}]->("Карманова Кира Петровна")          |
|      ("Пенюшин Николай Валентинович")-[:event@0{event_id: 658710}]->("Ряполовская Лилия Павловна")      |
|         ("Пакшина Римма Владимировна")-[:event@0{event_id: 799057}]->("Рявкина Лидия Ефимовна")         |
|       ("Башнина Антонина Г

In [13]:
session.execute('MATCH ()-[e:event]-() WHERE dst(e) == src(e) RETURN e')

ResultSet(keys: ['e'], values: )

Петель нет, но некоторые события повторяются, выведем их.

In [14]:
res = result_to_df(session.execute('MATCH ()-[e:event]-() \
                            WITH e.event_id as eid, count(e.event_id)/2 as cnt \
                            WHERE cnt >= 2 return eid, cnt \
                            ORDER BY cnt DESC'))

rep = res.iloc[:,0].to_list()

print_resp(session.execute(f'MATCH ()-[e:event]-() \
                            WHERE e.event_id in {rep} \
                            RETURN distinct e as event, e.event_id as id \
                            ORDER BY id'))

+--------------------------------------------------------------------------------------------------+--------+
|                                              event                                               |   id   |
+--------------------------------------------------------------------------------------------------+--------+
|   ("Яшина Полина Евгеньевна")-[:event@0{event_id: 70049}]->("Герасимовская Ксения Дамировна")    | 70049  |
|   ("Федова Анжелика Вадимовна")-[:event@0{event_id: 70049}]->("Вальдовский Альберт Ефимович")    | 70049  |
|    ("Кучеренко Ирина Ильинична")-[:event@0{event_id: 92995}]->("Болтик Григорий Максимович")     | 92995  |
|     ("Журик Альберт Евгеньевич")-[:event@0{event_id: 92995}]->("Бадьянова Римма Максимовна")     | 92995  |
|    ("Волынский Кирилл Федорович")-[:event@0{event_id: 117280}]->("Гайсумов Виктор Тимурович")    | 117280 |
|    ("Уточкин Евгений Анатольевич")-[:event@0{event_id: 117280}]->("Каганович Лилия Петровна")    | 117280 |
|   ("Бужа

Как видно, в основном участники образуют в основном группы по два, хотя иногда бывают и более крупные объединения.
![Alt text](src/Image%20(8).png)

Попробуем вывести все сообщества, где более одной связи.

In [15]:
from nebula3.data.DataObject import Relationship, ValueWrapper

# Введём список сложных сообществ
clubs = []
# Выбираем списки всех путей длинее 2х ребёр, сгрупированные по начальным вершинам
res = result_to_df(session.execute('MATCH (v)-[e:event*2..]-() RETURN id(v), collect(e)'))
rows = res.shape[0]
# Проходимся по группам
for row in range(rows):
    club = set()
    # По путям в группах
    for path in res.iloc[row,1]:
        for edge in path:
            # Достаём начало и конец и добавляем в текущее сообщество из тех,
            # кто в принципе есть в путях от этой вершины
            start = ValueWrapper.as_string(Relationship.start_vertex_id(edge))
            end = ValueWrapper.as_string(Relationship.end_vertex_id(edge))
            club.add(start)
            club.add(end)
    # Дополним имеющееся в списке сообщество, если есть пересечения с текущим
    flag = True
    for c in clubs:
        for p in club:
            if p in c:
                c.union(club)
                flag = False
                break
        if not flag:
            break
    # Или добавим целиком если пересечений нет
    if flag:
        clubs.append(club)

for c in clubs:
    print(c)
    print("Участников", len(c))

{'Яцкой Роберт Ильдарович', 'Поскребышев Яков Дмитриевич', 'Недовесков Владимир Иванович', 'Подолян Владислав Денисович', 'Майлина Гульнара Ивановна', 'Каехтин Ильдар Эдуардович'}
Участников 6
{'Сарсадских Алена Геннадьевна', 'Думлер Людмила Вячеславовна', 'Музалевская Ангелина Федоровна', 'Чикирева Мария Романовна', 'Аксанова Кристина Григорьевна', 'Шальнова Ольга Владимировна', 'Чечин Рамиль Константинович', 'Ковшов Глеб Германович', 'Тяжлов Ринат Владиславович', 'Домогаров Антон Максимович', 'Арбачаков Филипп Андреевич', 'Толкунова Валентина Маратовна', 'Андриевская Марина Ринатовна', 'Щенников Дмитрий Григорьевич', 'Абаренов Ильдар Робертович', 'Бодрякова Евгения Яновна', 'Ящукова Любовь Ефимовна', 'Яникеев Вячеслав Русланович', 'Бобрецова Светлана Артемовна', 'Вохменцев Владимир Владиславович', 'Алипичев Евгений Тимурович', 'Жубанов Анатолий Иванович', 'Урманцева Евгения Олеговна', 'Соломеина Кристина Георгиевна', 'Бордачев Никита Васильевич', 'Камилов Дамир Павлович', 'Нетужилова

![Alt text](src/Image%20(9).png)

Как видно, имееются несколько типов сложных сообществ: три собранных вокруг одного человека, одно в виде кольца и два с более сложной структурой из двух объединённых подсообществ.

Посмотрим кто из учасников имеет больше всего связей.

In [16]:
print_resp(session.execute('MATCH (p:person)-[e:event]-() \
                           RETURN p.person.name as person, count(e) as events \
                           ORDER BY events DESC LIMIT 30;'))

+-----------------------------------+--------+
|               person              | events |
+-----------------------------------+--------+
|      Ахромеева Алина Ивановна     |   50   |
|     Башнина Антонина Глебовна     |   14   |
|     Медведева Дарья Алексеевна    |   6    |
|     Диомидов Игорь Ильдарович     |   5    |
|     Зимнухова Карина Даниловна    |   5    |
|      Шолохов Игорь Робертович     |   4    |
|  Двигубская Валентина Геннадьевна |   3    |
|      Пафомова Кира Вадимовна      |   3    |
|    Недовесков Владимир Иванович   |   2    |
|     Каехтин Ильдар Эдуардович     |   2    |
|      Троекуров Глеб Ефимович      |   2    |
|    Даниленко Владимир Семенович   |   2    |
|    Радионова Тамара Ярославовна   |   2    |
|    Подолян Владислав Денисович    |   2    |
|    Поскребышев Яков Дмитриевич    |   2    |
|     Дорожкин Анатолий Егорович    |   2    |
|    Мараховская Дарья Романовна    |   2    |
|   Батиевская Ангелина Романовна   |   2    |
|     Бугайчу

Очевидно первые три участника с наибольшим количеством связей являются центрами своих сообществ. 
![Alt text](src/Image%20(12).png)

Попробуем вывести для них связи где они являются первым участником события и где вторым.

In [17]:
print_resp(session.execute('MATCH (p)-[e:event]->() \
                            WHERE id(p) in ["Ахромеева Алина Ивановна", "Башнина Антонина Глебовна", "Медведева Дарья Алексеевна"] \
                            RETURN id(p) as person, count(e) as first;'))
print_resp(session.execute('MATCH (p)<-[e:event]-() \
                            WHERE id(p) in ["Ахромеева Алина Ивановна", "Башнина Антонина Глебовна", "Медведева Дарья Алексеевна"] \
                            RETURN id(p) as person, count(e) as second;'))

+----------------------------+-------+
|           person           | first |
+----------------------------+-------+
|  Ахромеева Алина Ивановна  |   49  |
| Медведева Дарья Алексеевна |   1   |
| Башнина Антонина Глебовна  |   14  |
+----------------------------+-------+
+----------------------------+--------+
|           person           | second |
+----------------------------+--------+
|  Ахромеева Алина Ивановна  |   1    |
| Медведева Дарья Алексеевна |   5    |
+----------------------------+--------+


Тут можно видеть некоторую странность, Ахромеева Алина Ивановна является вторым участником события только в одном случае, Медведева Дарья Алексеевна первым только в одном случае. Посмотрим что это за события.

In [18]:
print_resp(session.execute('MATCH (p2:person {name:"Ахромеева Алина Ивановна"})<-[e2:event]-() \
                            MATCH (p3:person {name:"Медведева Дарья Алексеевна"})-[e3:event]->() \
                            UNWIND [e2, e3] as e \
                            RETURN e as strange_event'))

+-----------------------------------------------------------------------------------------+
|                                      strange_event                                      |
+-----------------------------------------------------------------------------------------+
|  ("Ошуров Павел Ильдарович")-[:event@0{event_id: 800116}]->("Ахромеева Алина Ивановна") |
| ("Медведева Дарья Алексеевна")-[:event@0{event_id: 173973}]->("Глазков Артур Петрович") |
+-----------------------------------------------------------------------------------------+


Теперь выведем более сложные сообщества.

In [20]:
print_resp(session.execute(
    'MATCH ()-[e:event*3..]-() \
    RETURN distinct e AS event, size(e) as len \
    ORDER BY len LIMIT 30'))

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|                                                                                                                                                 event                                                                                                                                                 | len |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|              [("Нагайцева Анжелика Яновна")-[:event@0{event_id: 410960}]->("Троекуров 

![Alt text](src/Image%20(10).png)

Закроем сессию и остановим контейнеры.

In [21]:
# release session
session.release()
# close the pool
connection_pool.close()
!docker-compose --project-directory nebula-graph-studio/deployment/docker down
!docker-compose --project-directory nebula-docker-compose down

Container docker-web-1  Stopping
Container docker-web-1  Stopping
Container docker-web-1  Stopped
Container docker-web-1  Removing
Container docker-web-1  Removed
Network docker_nebula-web  Removing
Network docker_nebula-web  Removed
Container nebula-docker-compose-graphd2-1  Stopping
Container nebula-docker-compose-graphd2-1  Stopping
Container nebula-docker-compose-console-1  Stopping
Container nebula-docker-compose-console-1  Stopping
Container nebula-docker-compose-graphd1-1  Stopping
Container nebula-docker-compose-graphd1-1  Stopping
Container nebula-docker-compose-graphd1-1  Stopped
Container nebula-docker-compose-graphd1-1  Removing
Container nebula-docker-compose-graphd1-1  Removed
Container nebula-docker-compose-graphd2-1  Stopped
Container nebula-docker-compose-graphd2-1  Removing
Container nebula-docker-compose-graphd2-1  Removed
Container nebula-docker-compose-console-1  Stopped
Container nebula-docker-compose-console-1  Removing
Container nebula-docker-compose-console-1  